## Overview
## Outline
1. Import Library
2. Setup pre-requisites
3. Extract dataset to images
4. Upload images to Azure data store
4. Setting up Azure ML Infrastructure

### 1. Import library

In [5]:
## Install needed packages
%pip install opencv-python torch torchvision torchaudio azureml-core

  Obtaining dependency information for azureml-core from https://files.pythonhosted.org/packages/e5/da/ed67a11a1baa8dff268ce71965fb936cd39e244ddd82134c3325f89fa7a8/azureml_core-1.53.0-py3-none-any.whl.metadata
  Using cached azureml_core-1.53.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached backports.tempfile-1.0-py2.py3-none-any.whl (4.4 kB)
  Obtaining dependency information for pathspec<1.0.0 from https://files.pythonhosted.org/packages/b4/2a/9b1be29146139ef459188f5e420a66e835dda921208db600b7037093891f/pathspec-0.11.2-py3-none-any.whl.metadata
  Using cached pathspec-0.11.2-py3-none-any.whl.metadata (19 kB)
  Obtaining dependency information for msal<2.0.0,>=1.15.0 from https://files.pythonhosted.org/packages/35/33/0fd933b627879a9855d02a83a57929b45d0bdbeb050ddd63109cc404fbf6/msal-1.24.1-py2.py3-none-any.whl.metadata
  Using cached msal-1.24.1-py2.py3-none-any.whl.metadata (11 kB)
  Using cached msal_extensions-1.0.0-py2.py3-none-any.whl (19 kB)
  Using cached knack-0.10.1-py3-non

In [1]:
import os
from importlib import reload

import cv2
from azureml.core import Workspace, Dataset, Datastore

## Using torchvision to create a dataset
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torchvision
import torch


### 2. Setup pre-requisites

In [2]:
## Setup Azure ML Workspace
ws = Workspace.from_config()
## From workspace, get/create the default datastore
ds = ws.get_default_datastore()
ws, ds

(Workspace.create(name='ws-vunn-iusai-sea-sp6k7', subscription_id='d554f489-6933-4c33-8722-a536b3682bd7', resource_group='rg-vunn-iusai-sea-sp6k7'),
 {
   "name": "workspaceblobstore",
   "container_name": "azureml-blobstore-5fddeec9-0fbc-4996-b95f-26996d3f3bcd",
   "account_name": "wsvunniusaisea7140421596",
   "protocol": "https",
   "endpoint": "core.windows.net"
 })

### 3. Extract dataset to images

In [3]:
## define videos location + images output location
video_path = 'videos/'
images_path = 'images/'
print('video_path: ', video_path)
print('images_path: ', images_path)

video_path:  videos/
images_path:  images/


In [4]:
import v2i
reload(v2i)
from v2i import extract_images_from_videos
import threading

def extract_images_from_videos_collection(video_path, images_path):
    ## get all videos file in video_path
    video_list_location_collection = os.listdir(video_path)

    ## define total_label_dict
    ## create threading pool
    thread = []
    ## for each video file
    for video_list_location in video_list_location_collection:
        ## check if video_location is not a directory (i.e. is a file), then skip
        if not os.path.isdir(video_path + video_list_location):
            continue
        ## list videos in video_location
        thread.append(threading.Thread(target=extract_images_from_videos, args=(video_path + video_list_location, images_path, 1)))
        # extract_images_from_videos(video_path + video_list_location, images_path, inteval=1)

    ## start all threads
    for t in thread: t.start()
    ## wait for all threads to finish
    for t in thread:
        t.join()
    
    ## check images in images_path
    images_list_location_collection = os.listdir(images_path)
    print('images_list_location_collection: ', images_list_location_collection)

# video_subject_6_path = video_path + '/subject6/'
# ## list videos in video_location
# label_dict = extract_images_from_videos(video_subject_6_path, images_path)

## extract images from videos
extract_images_from_videos_collection(video_path, images_path)


Label dict of videos/subject10/huangpi21.avi (start): [['0', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject11/alhashe31.avi (start): [['0', 0], ['2', 0]]
Label dict of videos/subject1/Yousef1.avi (start): [['0', 0], ['1', 0], ['2', 0], ['5', 0], ['6', 0], ['3', 0]]
Label dict of videos/subject14/chuangy61.avi (start): [['0', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject13/guolingh1.avi (start): [['0', 0], ['6', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject12/makiluke1.avi (start): [['0', 0], ['2', 0]]
Label dict of videos/subject15/bajajpak1.avi (start): [['0', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject16/dhingra51.avi (start): [['0', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject19/wangyus11.avi (start): [['0', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject17/zhoulian1.avi (start): [['0', 0], ['1', 0], ['2', 0]]
Label dict of videos/subject18/yuqianyi1.avi (start): [['0', 0], ['2', 0], ['1', 0]]
Label dict of videos/subject21/Mark1.avi (start)

Exception in thread Thread-10 (extract_images_from_videos):
Traceback (most recent call last):
  File "c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\nnvuf\source\ai-final-project\v2i.py", line 67, in extract_images_from_videos
    cv2.destroyAllWindows()
cv2.error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1266: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'

Exception in thread Thread-5 (extract_images_from_videos):
Traceback (most recent call last):
  File "c:\Users\nnvuf\AppData\Local\Programs\

images_list_location_collection:  ['0', '1', '2', '3', '5', '6']


### 4. Upload images to Azure data store

In [6]:
## upload images to data asset using Dataset.File.upload_directory
Dataset.File.upload_directory(src_dir=images_path, target=(ds, 'images-extra-small'), overwrite=True, show_progress=True) \
    .register(workspace=ws, name='images-extra-small', description='images-extra-small') ## register dataset \\


Validating arguments.
Arguments validated.
'overwrite' is set to True. Any file already present in the target will be overwritten.
Uploading files from 'c:/Users/nnvuf/source/ai-final-project/images' to 'images-extra-small'
Creating new dataset


{
  "source": [
    "('workspaceblobstore', '/images-extra-small')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "22c174ee-e3e7-4e60-a949-e20eb237c73d",
    "name": "images-extra-small",
    "version": 1,
    "description": "images-extra-small",
    "workspace": "Workspace.create(name='ws-vunn-iusai-sea-sp6k7', subscription_id='d554f489-6933-4c33-8722-a536b3682bd7', resource_group='rg-vunn-iusai-sea-sp6k7')"
  }
}

### 5. Setup public workspace endpoint

In [9]:
dataset_collection_list = ['images-extra-small', 'images-small-v2', 'images-medium']
## register dataset using Dataset.File.from_files
for dataset_collection in dataset_collection_list:
    ## check if dataset_collection is already registered
    if dataset_collection in ws.datasets.keys():
        print('dataset_collection: ', dataset_collection, ' is already registered')
        continue
    Dataset.File.from_files(path=(ds, dataset_collection)) \
        .register(workspace=ws, name=dataset_collection, description=dataset_collection) ## register dataset \\


dataset_collection:  images-extra-small  is already registered
dataset_collection:  images-small-v2  is already registered
dataset_collection:  images-medium  is already registered


## Define and Train models
1. ResNET
2. DenseNET

### Setup device + load dataset

In [29]:
## import train and scrore function from train.py
import train as t;
reload(t)
from train import train, score_model, get_device

device = get_device()
device

'cuda:0'

In [30]:
# download data asset to local if needed
images_path = 'images'
images_ds_path = "images-extra-small"
if not os.path.exists(images_path):
    os.mkdir(images_path)
# download data asset to local if images_path is empty
if len(os.listdir(images_path)) == 0:
    print('images_path is empty, download images_ds to images_path')
    images_ds = Dataset.File.from_files(path=(ds, images_ds_path)) \
        .register(workspace=ws, name=images_ds_path, description=images_ds_path)
    images_ds.download(target_path=images_path, overwrite=True)

#### Prepare Dataset

In [31]:
from torch.utils.data import DataLoader
import data_set as ds
reload(ds)

transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
imageDataset = ds.ImageDataset('images', transform=transform)
## define batch_size
batch_size = 64

In [32]:
image, label = imageDataset.get_image(0)
# image.show()
labels = imageDataset.labels()
## show labels in Interger
print('labels: ',  [int(l) for l in labels])

labels:  [0, 1, 2, 3, 5, 6]


In [33]:
## split dataset into train and test dataset using random_split
from torch.utils.data import random_split
train_val_size = int(0.9 * len(imageDataset))
test_size = len(imageDataset) - train_val_size
train_val_ds, test_ds = random_split(imageDataset, [train_val_size, test_size])

## split train_val_ds into train_ds and val_ds using random_split
train_size = int(0.8 * len(train_val_ds))
val_size = len(train_val_ds) - train_size
train_ds, val_ds = random_split(train_val_ds, [train_size, val_size])

train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_ds, batch_size=batch_size, shuffle=True, num_workers=0)
test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True, num_workers=0)

print('train_ds: ', len(train_ds))
print('val_ds: ', len(val_ds))
print('test_ds: ', len(test_ds))


train_ds:  37715
val_ds:  9429
test_ds:  5239


### 1. Resnes18

In [34]:
## get resnet model of image classification from torchvision
model = torchvision.models.resnet18(pretrained=True)
## define optimizer using Adam and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer, loss_fn

(Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 ),
 CrossEntropyLoss())

In [35]:
## train model
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=1, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

KeyboardInterrupt: 

In [ ]:
## train model using device (CPU or GPU) 10 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=10, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

Epoch 1/10, Train Loss: 0.2793, Test Loss: 0.3115
Epoch 2/10, Train Loss: 0.1936, Test Loss: 0.2297
Epoch 3/10, Train Loss: 0.1382, Test Loss: 0.2441
Epoch 4/10, Train Loss: 0.1056, Test Loss: 0.2602
Epoch 5/10, Train Loss: 0.0861, Test Loss: 0.2807
Epoch 6/10, Train Loss: 0.0804, Test Loss: 0.2235
Epoch 7/10, Train Loss: 0.0589, Test Loss: 0.2136
Epoch 8/10, Train Loss: 0.0591, Test Loss: 0.2263
Epoch 9/10, Train Loss: 0.0509, Test Loss: 0.2329
Epoch 10/10, Train Loss: 0.0417, Test Loss: 0.2341


0.23411470530744863

In [17]:
score_model(model, loss_fn, val_dataloader, device=device)

0.24655256064399098

In [ ]:
## save model
torch.save(model.state_dict(), 'model_resnes18.small.pth')

### 2. DenseNet

In [20]:
## get denseNet model of image classification from torchvision
model = torchvision.models.densenet121(pretrained=True)
## define optimizer using Adam and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer, loss_fn

(Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 ),
 CrossEntropyLoss())

In [21]:
## train model using device (CPU or GPU) 1 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=1, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

KeyboardInterrupt: 

In [ ]:
## train model using device (CPU or GPU) 10 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=10, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

In [ ]:
## save model
torch.save(model.state_dict(), 'model_densenet121.pth')

### 3. Using Yolov5 to extract face then using ResNet to classification the output

In [ ]:
## use yoloresnet model
import model
reload(model)
from model import YoloResnet
model = YoloResnet()
## define optimizer using Adam and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer, loss_fn

In [ ]:
## train model using device (CPU or GPU) 1 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=1, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

In [ ]:
## train model using device (CPU or GPU) 10 epoch
train(model, optimizer, loss_fn, train_dataloader, val_dataloader, epochs=10, device=device)
## score model
score_model(model, loss_fn, test_dataloader, device=device)

In [ ]:
## save model
torch.save(model.state_dict(), 'model_yoloresnet.pth')